In [5]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import pandas as pd
import torch
import glob
import pickle
import matplotlib.pyplot as plt

from interpretation.interpret import compute_importance_score_c_type, compute_importance_score_bias, visualize_sequence_imp
from models.models import CATAC2, CATAC_w_bias

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Sample from peak sequences

In [2]:
with open('../results/peaks_seq.pkl', 'rb') as file:
    seq = pickle.load(file).sequence

# Compute importance scores

In [12]:
path_model = '../results/train_res/128_model.pkl'
path_seq = '../results/synthetic_results/synthetic_sequences.pkl'

all_c_type = ['Immature', 'Mesenchymal', 'Myoblast', 'Myogenic', 'Neuroblast',
       'Neuronal', 'Somite']
time_point = ["D8", "D12", "D20", "D22"]

#Load the model
model = CATAC_w_bias(nb_conv=8, nb_filters=128, first_kernel=21, 
                      rest_kernel=3, out_pred_len=1024, 
                      nb_pred=4)
        
model.load_state_dict(torch.load(path_model, map_location=torch.device('cpu')))

path_model_bias = "../data/Tn5_NN_model.h5"

#Compute attribution scores
seq, shap_scores, proj_scores = compute_importance_score_bias(model, path_model_bias, seq, device, "Somite", all_c_type, 1)

/Users/adorman/Documents/master-thesis/src/interpretation/interpret.py:186: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  c_type = c_type.tile((seq[0].shape[0],1))
/Users/adorman/Documents/master-thesis/src/interpretation/interpret.py:193: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  shap_scores = [compute_shap_score_bias(model,s,torch.from_numpy(background[i]), torch.from_numpy(tn5_bias[i]), idx_time) for i,s in enumerate(seq)]


torch.Size([20, 1])


/Applications/anaconda3/envs/LeKira/lib/python3.10/site-packages/torch/nn/modules/module.py:1352: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])
torch.Size([20, 1])


In [ ]:
#Save encoded seq + scores
np.savez('../results/encod_seq.npz', seq[:,:4,:])
np.savez('../results/seq_scores.npz', shap_scores[:,:4,:], proj_scores[:,:4,:])

print("Shap scores saved!")

In [ ]:
seq = np.load('../results/encod_seq.npz')["arr_0"]
shap_scores = np.load('../results/seq_scores.npz')
proj_scores = shap_scores['arr_1']; shap_scores = shap_scores['arr_0']

# Visualize few examples

In [ ]:
visualize_sequence_imp(proj_scores[[73],:4,:] ,0, 4096)
visualize_sequence_imp(proj_scores[[1266],:4,:] ,0, 4096)
visualize_sequence_imp(proj_scores[[563],:4,:] ,0, 4096)

# Use TF-modisco to find TFBS
Following tutorial at: https://github.com/jmschrei/tfmodisco-lite/blob/main/examples/ModiscoDemonstration.ipynb

In [ ]:
!modisco motifs -s  ../results/encod_seq.npz -a  ../results/seq_scores.npz -n 2000 -o modisco_results.h5

In [ ]:
!modisco report -i modisco_results.h5 -o report/

In [ ]:
from IPython.display import HTML
HTML('report/motifs.html')

# Run TOMTOM on modisco results

In [ ]:
!modisco report -i modisco_results.h5 -o report/TOMTOM -s report/TOMTOM -m ../data/JASPAR_motif.txt

In [ ]:
from IPython.display import HTML

HTML('report/motifs.html')